In [170]:
#Import Dependencies

from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np

from time import sleep
import re

In [171]:
# Sets a path to Google Chrome
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/joescuteri/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [185]:
# Root URL of the page to be scraped
url = 'https://twitchtracker.com/games'
# Visit the URL
browser.visit(url)
# Parse the link using BeautifulSoup
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [186]:
# Find the games and their respective views from page 1 of the table
href_tags = soup.find_all(href=True)
raw_games = href_tags[40:60]

In [187]:
tags = []
for x in raw_games:
    tags.append(str(x))

In [188]:
games = []
for element in tags:
    if element not in games:
        front = element.split(">")[1]
        back = front.split("<")[0]
        games.append(back)

In [189]:
viewhtml = soup.find_all('div', class_="ri-value")
cleanedviews = []
for x in viewhtml:
    cleanedviews.append(str(x))

In [190]:
views = []
for element in cleanedviews:
    if element not in views:
        front = element.split(">")[2]
        back = front.split("<")[0]
        views.append(back)

In [191]:
# Upload to DataFrame 
df = pd.DataFrame(games, columns = ['GAME'])
df['VIEWS'] = views
df

,GAME,VIEWS
0,Just Chatting,376K
1,League of Legends,285K
2,Grand Theft Auto V,211K
3,VALORANT,128K
4,Rust,122K
5,Fortnite,93.2K
6,Apex Legends,91.7K
7,Escape from Tarkov,72.9K
8,Dota 2,64.9K
9,Call of Duty: Warzone,57.7K


In [193]:
# Change VIEWS column to integers

df.VIEWS = (df.VIEWS.replace(r'[KM]+$', '', regex=True).astype(float) * df.VIEWS.str.extract(r'[\d\.]+([KM]+)', expand=False)
             .fillna(1)
             .replace(['K','M'], [10**3, 10**6]).astype(int))

In [194]:
avg_views = df['VIEWS'].mean()
median_views = df['VIEWS'].median()

In [195]:
median_views

54850.0

In [11]:
# Convert DataFrame into Dictionary
twitch = pd.Series(df3.VIEWS.values,index=df3.GAMES).to_dict()

In [12]:
twitch

{'Just Chatting': '374,034',
 'League of Legends': '269,528',
 'Rust': '223,036',
 'Grand Theft Auto V': '198,498',
 'Escape from Tarkov': '122,840',
 'VALORANT': '116,776',
 'Apex Legends': '96,378',
 'Fortnite': '89,392',
 'Dota 2': '71,811'}

# Migrating it to MongoDB

In [26]:
# connecting to mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# declare the database
db = client.twitchviews_db
collection = db.items

In [29]:
# Push into MongoDB
collection.insert_one(twitch)